In [2]:
from YanAPI import YanAPI
from mutagen.mp3 import MP3
import time
from find_music import find_music
from download_music import download_music

ip_addr = "192.168.1.12" # please change to your yanshee robot IP
YanAPI.yan_api_init(ip_addr)


ImportError: cannot import name 'YanAPI'

In [ ]:
def listen():
    while True:
        YanAPI.start_voice_tts("What do want to listen to",False)
        time.sleep(2) #sleep 2 seconds to ensure the voice will not be interapt by the code below
        listen_res = YanAPI.sync_do_voice_asr_value() # listen again to get the name.
        if len(listen_res["answer"]) > 0:# user said something
            YanAPI.start_voice_tts("Please wait while I find the song",False)
            time.sleep(3)
            result = find_music(listen_res['answer'])
            YanAPI.start_voice_tts("Is this what you are looking for: "+result+"",False)
            if YanAPI.sync_do_voice_asr_value()['answer'] == 'no':
                continue
            else:
                YanAPI.start_voice_tts("Please wait while I download the song",False)
                break
        else:
            YanAPI.start_voice_tts("I cannot hear you, try again",False)
    return listen_res['answer']

In [ ]:
def play_song():
    title = download_music(listen())
    YanAPI.start_play_music(f'./Music/{title}')
    time.sleep(5)
    # time.sleep(MP3(f'./Music/{title}').info.length)
    YanAPI.stop_play_music()

In [ ]:
play_song()